# VacationPy
----

## Analysis



##### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
city_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hobart,-42.8794,147.3294,69.80,30,0,19.57,AU,1613960372
1,1,auki,-8.7676,160.7034,83.98,79,94,4.14,SB,1613960387
2,2,vaini,-21.2000,-175.2000,82.40,69,40,13.80,TO,1613960389
3,3,ushuaia,-54.8000,-68.3000,41.00,81,75,6.91,AR,1613960392
4,4,neuquen,-38.9516,-68.0591,69.80,40,0,9.22,AR,1613960394


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [20]:
# combine lat and long to be used as locations
location = city_df[["Lat", "Lng"]].astype(float)

# store humidity so it can be used as the weight in the heat map
humidity = city_df["Humidity"]

In [37]:
# create the gmaps figure
fig_layout = {'width': '900px', 'height': '550px'}
fig = gmaps.figure(center= [27.357072, -25.565121], zoom_level = 1.75, layout = fig_layout)

# create heat map layer
heat_layer = gmaps.heatmap_layer(location, weights = humidity, dissipating = False, 
                                 max_intensity=500, point_radius = 3)

# add heat layer to map
fig.add_layer(heat_layer)

# show map
fig

Figure(layout=FigureLayout(height='550px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
viable_cities = city_df.loc[(city_df["Max Temp"] < 80)\
                            & (city_df["Max Temp"] > 70)\
                            & (city_df["Wind Speed"] < 10)\
                            & (city_df["Cloudiness"]==0)
                           ].dropna()

viable_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,saint-philippe,-21.3585,55.7679,77.00,69,0,3.44,RE,1613960422
52,52,yamoussoukro,6.8206,-5.2767,75.20,83,0,4.61,CI,1613960501
89,89,dorado,18.4588,-66.2677,77.00,86,0,1.99,PR,1613960419
165,165,tezu,27.9167,96.1667,74.39,25,0,1.23,IN,1613960752
166,166,busselton,-33.6500,115.3333,75.00,60,0,5.01,AU,1613960754


In [8]:
viable_cities.describe()

,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.0,30.00000,3.000000e+01
mean,491.833333,5.686607,17.822540,74.003667,59.566667,0.0,4.66500,1.613962e+09
std,277.372410,22.393091,76.385559,2.354588,24.717275,0.0,2.15684,6.693293e+02
min,17.000000,-34.666700,-102.233300,70.830000,17.000000,0.0,1.01000,1.613960e+09
25%,291.000000,-18.559000,-54.627325,71.840000,38.000000,0.0,3.44000,1.613961e+09
50%,474.000000,17.062500,49.779050,73.640000,67.500000,0.0,4.61000,1.613962e+09
75%,739.750000,23.046725,75.258325,75.150000,80.750000,0.0,6.27500,1.613962e+09
max,949.000000,31.776500,115.716700,78.800000,94.000000,0.0,9.46000,1.613963e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Save relevate columns into a new dataframe called hotel_df
hotel_df = viable_cities[["City", "Lat", "Lng", "Country"]].copy()

# add a column name called Hotel Name
hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Lat,Lng,Country,Hotel Name
17,saint-philippe,-21.3585,55.7679,RE,
52,yamoussoukro,6.8206,-5.2767,CI,
89,dorado,18.4588,-66.2677,PR,
165,tezu,27.9167,96.1667,IN,
166,busselton,-33.6500,115.3333,AU,


In [14]:
# set the parameters for the hotel search
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# for more information on nearby search and it's parameters: 
# https://developers.google.com/places/web-service/search
# https://developers.google.com/places/web-service/supported_types

for index, row in hotel_df.iterrows():
    
    # get lat and lng
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add location to params
    params["location"]= f'{lat},{lng}'
    
    # set the base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make a request & store as json
    response = requests.get(base_url, params).json()
    hotel_name = response["results"]
    
    # store the first result in the hotel_df
    
    try:
        print(f'Updating {row["City"]}...')
        hotel_df.loc[index, "Hotel Name"] = hotel_name[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    time.sleep(2) # Sleep for 5 seconds

print("--Done--")    

Updating saint-philippe...
Updating yamoussoukro...
Updating dorado...
Updating tezu...
Updating busselton...
Updating saint-pierre...
Updating victoria...
Updating harda...
Updating salvador...
Updating abu dhabi...
Updating bandrele...
Updating maldonado...
Updating bousse...
Updating mundra...
Updating pasni...
Updating ziniare...
Updating nanzhang...
Updating tres lagoas...
Updating ribas do rio pardo...
Updating jati...
Missing field/result... skipping.
Updating concordia...
Updating shilong...
Updating ormara...
Updating la orilla...
Updating salina cruz...
Updating australind...
Updating myitkyina...
Updating maragogi...
Updating guanica...
Updating lazaro cardenas...
--Done--


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
